# <center> Monthly Data for Finance Team regarding Application count, Booked Count </center>

In [1]:
import pandas as pd
import numpy as np 


from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project='prj-prod-dataplatform')

In [2]:
sq = """
SELECT a.loanaccountnumber, a.new_loan_type as Product, Extract(date from a.disbursementDateTime) as BookedDt, a.disbursementDateTime, a.disbursedLoanAmount, a.disbursedLoanTenur as Term,
f.principalAmount,
f.interestAmount,
f.scheduleChargeAmount as VasFees,
f.repaymentAmount
FROM
prj-prod-dataplatform.risk_credit_mis.loan_master_table a
join prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application e
on e.loanAccountNumber = a.loanAccountNumber
join prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_repayment_schedule  f
on a.loanAccountNumber = f.loanId and f.repaymentNumber = 1
WHERE
DATE_TRUNC(a.disbursementDateTime,DAY) between DATE '2023-10-05' and CURRENT_DATE - 1
and e.vas_flag = 'true'
ORDER BY 3;
"""

In [3]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 27d5d192-6e14-4f83-ae95-c3dd41e846a0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [4]:
print(f"The rows and columns exported are:\t{df.shape}")

The rows and columns exported are:	(14612, 10)


In [5]:
pd.set_option("Display.max_columns", None)
df.head()

,loanaccountnumber,Product,BookedDt,disbursementDateTime,disbursedLoanAmount,Term,principalAmount,interestAmount,VasFees,repaymentAmount
0,60822612480014,SIL-Instore,2023-10-05,2023-10-05 18:06:28,14319.0,9,1313.22,656.71,179.46,2149.39
1,60822608950015,SIL-Instore,2023-10-05,2023-10-05 14:03:26,5459.0,6,808.01,250.37,96.42,1154.80
2,60822610350013,SIL-Instore,2023-10-05,2023-10-05 19:59:57,14995.0,12,956.72,687.72,149.81,1794.25
3,60822612380012,SIL-Instore,2023-10-05,2023-10-05 18:40:26,7408.0,12,472.66,339.75,74.01,886.42
4,60822610570019,SIL-Instore,2023-10-05,2023-10-05 15:55:13,10299.0,9,944.54,472.34,129.08,1545.96


In [6]:
sq = """
-- Set your date range
DECLARE start_date DATE DEFAULT DATE '2022-01-01';
DECLARE end_date DATE DEFAULT DATE '2024-01-31';

-- Create a temporary table with all dates within the specified range
WITH temp_dates AS (
  SELECT
    DATE_ADD(start_date, INTERVAL n DAY) AS date
  FROM
    UNNEST(GENERATE_ARRAY(0, DATE_DIFF(date(current_date()), start_date, DAY))) AS n
)

-- Use the temporary table in your main query
SELECT
  td.date,
  format_date('%Y-%m', td.date) month,
  lmt.loanType original_product_type,
  lmt.new_loan_type new_product_type,
  count(distinct case when date(lmt.startApplyDateTime) = td.date then lmt.digitalLoanAccountId end) cnt_application,
  count(distinct case when date(lmt.decision_date) = td.date then lmt.digitalLoanAccountId end) Application_decisioned,
  count(distinct case when date(lmt.decision_date) = td.date and date(lmt.approvedDateTime) = td.date then lmt.digitalLoanAccountId end) cnt_Approved,
  count(distinct case when date(lmt.decision_date) = td.date and date(lmt.rejectedDateTime) = td.date then lmt.digitalLoanAccountId end) cnt_Rejected,
  count(distinct case when date(lmt.disbursementDateTime) = td.date and lmt.flagDisbursement = 1 then lmt.loanAccountNumber end) cnt_booked,
  sum(case when date(lmt.disbursementDateTime) = td.date then lmt.disbursedLoanAmount else 0 end) Booked_amount,
  count(distinct case when date(lmt.disbursementDateTime) = td.date and lmt.flagDisbursement = 1 and e.vas_flag = 'true' then lmt.loanAccountNumber end) SoldwithVas
FROM
`risk_credit_mis.loan_master_table` lmt, temp_dates td
left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application e on e.loanAccountNumber = lmt.loanAccountNumber
group by 1,2,3,4
order by 1 desc;
"""

In [7]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The rows and columns exported are:\t {df.shape}")

Job ID 8931bcb5-891c-4591-b970-b81a34fe74d2 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns exported are:	 (5341, 11)


In [8]:
df.head(10)

,date,month,original_product_type,new_product_type,cnt_application,Application_decisioned,cnt_Approved,cnt_Rejected,cnt_booked,Booked_amount,SoldwithVas
0,2024-02-02,2024-02,TSBL,Flex-up,0,0,0,0,0,0.00,0
1,2024-02-02,2024-02,TUL,Quick,71,47,0,47,0,0.00,0
2,2024-02-02,2024-02,BIGLOAN,Big Loan,4,0,0,0,0,0.00,0
3,2024-02-02,2024-02,FLEXUP,Flex-up,0,0,0,0,0,0.00,0
4,2024-02-02,2024-02,BNPL,SIL-Instore,0,0,0,0,0,0.00,0
5,2024-02-02,2024-02,TSBL,Flex,0,0,0,0,0,0.00,0
6,2024-02-02,2024-02,BNPL,SIL-Online(discontinued),0,0,0,0,0,0.00,0
7,2024-02-01,2024-02,TSBL,Flex-up,9,6,0,0,6,107489.41,0
8,2024-02-01,2024-02,BNPL,SIL-Online(discontinued),0,0,0,0,0,0.00,0
9,2024-02-01,2024-02,TUL,Quick,894,634,18,612,26,389000.00,10


In [9]:
df.to_excel(r"Applicationinfo_finance.xlsx", index = False)